In [2]:
!pip install pywikibot

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.6/706.6 KB 14.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 KB 29.7 MB/s eta 0:00:00


In [1]:
import pywikibot


def get_wikidata(wikipedia_title):
    try:
        site = pywikibot.Site("en", "wikipedia")  # Connect to the English Wikipedia
        page = pywikibot.Page(site, wikipedia_title)  # Get the Wikipedia page
        item = pywikibot.ItemPage.fromPage(page)  # Get the associated Wikidata item
        item.get()
        to_add = ""
        if "en" in item.descriptions:  # Fetch all data of the Wikidata item
            to_add += item.descriptions['en'] + " ; "# Return the English description
        added = 0
        for category in page.categories()[:3]:
            cat = category.title().split(":")[1]
            if "All articles" in cat or "Articles" in cat or "Pages" in cat or "Use" in cat or "CS1" in cat or "Webarchive" in cat or "Articles" in cat or "Pages" in cat or "Wikipedia" in cat:
                continue
            else:
                to_add += category.title() + " ; "
                added += 1
            if added == 3:
                break

    except:
        to_add

# wikipedia_title = "KEGG"
# attributes = get_wikidata(wikipedia_title)
# # print(dir(attributes))
# print(attributes.descriptions["en"])
# for key in attributes.text:
#     print(key + "\n")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [15]:
test = get_wikidata("Kegg Pipe Organ Builders")
print(test)

In [2]:
def enrich(training_item):
    candidates = training_item["question"].split("</ec>")
    enriched_candidates = []
    for candidate in candidates:
        if candidate != "" and candidate.strip() != 'NIL':
            if candidate != "":
                correct_candidate = candidate.strip()
                description = get_wikidata(correct_candidate)
                correct_candidate += f" : {description}"
                enriched_candidates.append(correct_candidate)
    context_string = ""
    for cand in enriched_candidates:
        context_string += cand + " </ec> "
    training_item["question"] = context_string
    return training_item


In [3]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from tqdm import tqdm

tokenized = []
file_path = "./aida_train_converted.jsonl"
from datasets import Dataset
from multiprocessing import Pool

original_data = []
if __name__ == "__main__":
    with open(file_path, "r") as file:
        for line in file:
            # Load each line as a JSON object
            data_line = json.loads(line)
            with Pool(10) as p:
                original_data = list(
                    tqdm(p.imap(enrich, data_line), total=len(data_line))
                )

    with open(f"{file_path}enriched.jsonl", "w") as f:
        for item in original_data:
            f.write(json.dumps(item) + "\n")

  0%|          | 0/18448 [00:00<?, ?it/s]Could not load cache: UnpicklingError('pickle data was truncated')
Could not load cache: EOFError('Ran out of input')
Could not load cache: EOFError('Ran out of input')
Could not load cache: EOFError('Ran out of input')
 13%|█▎        | 2310/18448 [1:22:23<11:55:25,  2.66s/it]ERROR: An error occurred for uri https://en.wikipedia.org/w/api.php?titles=English+people&inprop=protection&prop=info&action=query&indexpageids=&continue=&meta=userinfo&uiprop=blockinfo%7Chasmsg&maxlag=5&format=json
ERROR: Traceback (most recent call last):
  File "/home/rub/.local/lib/python3.10/site-packages/pywikibot/data/api/_requests.py", line 682, in _http_request
    response = http.request(self.site, uri=uri,
  File "/home/rub/.local/lib/python3.10/site-packages/pywikibot/comms/http.py", line 283, in request
    r = fetch(baseuri, headers=headers, **kwargs)
  File "/home/rub/.local/lib/python3.10/site-packages/pywikibot/comms/http.py", line 457, in fetch
    callbac

## Version for base dataset

In [1]:
def enrich_variant(sample):
  candidates = sample['candidates']
  enriched_c = []
  for candidate in candidates:
    if candidate != "" and candidate.strip() != 'NIL':
      
      correct_candidate = candidate.strip()
      description = get_wikidata(correct_candidate)
      correct_candidate += f" : {description}"
      enriched_c.append(correct_candidate)
  sample['candidates'] = enriched_c
  return sample

In [4]:
from multiprocessing import Pool
from tqdm import tqdm
import json

file_path = "./wiki-test-kilt.jsonl"
dataset = []
if __name__ == "__main__":
    with open(file_path, "r") as file:
        unfolded = []
        for line in file:
            # Load each line as a JSON object
            data_line = json.loads(line)

            unfolded.append(data_line)
        with Pool(10) as p:
            dataset = list(tqdm(p.imap(enrich_variant, unfolded), total=len(unfolded)))
    with open(f"{file_path}enriched.jsonl", "w") as f:
      for item in dataset:
          f.write(json.dumps(item) + "\n")

 41%|████      | 2788/6821 [1:27:13<56:58,  1.18it/s]  ERROR: An error occurred for uri https://en.wikipedia.org/w/api.php?titles=Greece+national+football+team&inprop=protection&prop=info&action=query&indexpageids=&continue=&meta=userinfo&uiprop=blockinfo%7Chasmsg&maxlag=5&format=json
ERROR: Traceback (most recent call last):
  File "/home/rub/.local/lib/python3.10/site-packages/pywikibot/data/api/_requests.py", line 682, in _http_request
    response = http.request(self.site, uri=uri,
  File "/home/rub/.local/lib/python3.10/site-packages/pywikibot/comms/http.py", line 283, in request
    r = fetch(baseuri, headers=headers, **kwargs)
  File "/home/rub/.local/lib/python3.10/site-packages/pywikibot/comms/http.py", line 457, in fetch
    callback(response)
  File "/home/rub/.local/lib/python3.10/site-packages/pywikibot/comms/http.py", line 343, in error_handling_callback
    raise response from None
  File "/home/rub/.local/lib/python3.10/site-packages/pywikibot/comms/http.py", line 448, 